In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
<div class="burk">
import argparse
import json
import os
import pickle

import numpy as np
import tensorflow as tf

from utils.plotting import _plot_element, plot_hist_simple, plot_scatter_simple, plot_line_simple
from database.MIMIC_discretized_exp import MIMIC_per_time_env_exp_rnn
from policy_training_and_evaluation.run_reward_estimator import set_up_model, set_up_data_loader

In [3]:
type = 7
trans_model_dir = '../models/trans_m_timepoint_obs_obs_normal0109-hl1-hu256-lr0.001-reg0.0001-kp0.7-bs64-n0-r7'
cache_dir = '../RL_exp_cache/1128-30mins-24hrs-20order-rnn-neg_sampled/state_mean_and_std.csv'

In [ ]:
def set_up_transition_model_and_data_loader(sess, dirname):
    with open(os.path.join(dirname, 'hyperparameters.pkl'), 'rb') as f:
        trans_model_hparams = pickle.load(f)
    with open(os.path.join(dirname, 'results.pkl'), 'rb') as f:
        trans_model_results = pickle.load(f)
    trans_model_variable_scope, trans_model, trans_model_process_exp_func, _ = \
        set_up_model(args=trans_model_hparams, type=trans_model_hparams.type, log=print)

    saver = \
        tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                  scope=trans_model_variable_scope))
    model_path = os.path.join(dirname, 'model-%d' % trans_model_results['best_model_idx'])
    saver.restore(sess=sess, save_path=model_path)
    return trans_model, trans_model_process_exp_func

In [ ]:
np.random.seed(0)
tf.set_random_seed(0)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    trans_model, trans_model_process_exp_func = set_up_transition_model(
        sess, args.trans_model_dir)
    get_data_loader = set_up_data_loader(sess=sess, type=type, cache_dir=cache_dir,
                                             train_batch_size=args.train_batch_size,
                                             test_batch_size=args.test_batch_size)